Allows 

In [1]:
import sys, os

import numpy as np
from importlib import reload

import grimsel.core.model_loop as model_loop
from grimsel.core.model_base import ModelBase as MB

import grimsel.auxiliary.aux_sql_func as aql
import grimsel.analysis.sql_analysis_comp as sql_analysis_comp

import grimsel.config as config
import grimsel.model_loop_modifier as model_loop_modifier

db = config.DATABASE

sc_out = 'out_replace_all'



module 'grimsel.config_local' has no attribute 'PATH_CSV'
Using default csv path /mnt/data/Dropbox/SHARED_REPOS/grimsel/input_data


In [2]:
connect_dict = dict(db=db,
                    password=config.PSQL_PASSWORD,
                    user=config.PSQL_USER,
                    port=config.PSQL_PORT,
                    host=config.PSQL_HOST)
sqlc = aql.sql_connector(**connect_dict)

# additional kwargs for the model
mkwargs = {
           'slct_encar': ['EL'],
           'slct_node': ['AT0', 'IT0', 'DE0', 'CH0', 'FR0'],
           'nhours': 168*2,
           'verbose_solver': True,
           'constraint_groups': MB.get_constraint_groups(excl=['chp', 'ror'])
           }

# additional kwargs for the i/o
iokwargs = {'resume_loop': False,
            'autocomplete_curtailment': True}

nvr, nst = 30, 31
nsteps_default = [
                  ('swvr', nvr, np.arange),     # select vre share
                  ('swst', nst, np.arange),     # select storage capacity
                  ('swtc', 3, np.arange),       # select storage tech
                  ('swpt', 3, np.arange),       # select vre type
                  ('swyr', 5, np.arange),       # select meteo year
                  ('swco', 3, np.arange),       # select co2 emission price
                  ('swrc', 26, np.arange)]      # select ramping cost

mlkwargs = {
            'sc_out': sc_out,
            'db': db,
            'nsteps': nsteps_default,
            'sql_connector': sqlc,
            }

sc_out = mlkwargs['sc_out']

ml = model_loop.ModelLoop(**mlkwargs, mkwargs=mkwargs, iokwargs=iokwargs)


swst_max = nst
dict_st = {nst: st for nst, st in enumerate(list(np.linspace(0, 0.3, swst_max)))}
swvr_max = nvr - 1
dict_vre = {nvr: vr for nvr , vr
            in enumerate(['default'] + list(np.linspace(0, 0.7, swvr_max)))}



self.slct_encar=['EL']
self.slct_pp_type=[]
self.slct_node=['AT0', 'IT0', 'DE0', 'CH0', 'FR0']
self.nhours=336
self.constraint_groups=['capacity', 'charging_level', 'chp_new', 'energy_aggregation', 'energy_constraint', 'hydro', 'monthly_total', 'objective', 'ramp_rate', 'supply', 'transmission', 'variables', 'yearly_cost']
Output schema:  out_replace_all ; resume loop= False
Dropping output schema:  out_replace_all
Reading input table def_month 
Reading input table def_week 
Reading input table parameter_month 
Reading input table def_plant 
Reading input table def_node filtered by nd in ['AT0', 'IT0', 'DE0', 'CH0', 'FR0']
Reading input table def_pp_type 
Reading input table def_encar filtered by ca in ['EL']
Reading input table def_plant filtered by nd_id in [0, 1, 2, 3, 4], pt_id in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Reading input table profdmnd filtered by nd_id in [0, 1, 2, 3, 4], ca_id in [0]
Reading input table profchp filtered by nd_id in [0, 1, 2, 3, 4]
Read

/mnt/data/Dropbox/SHARED_REPOS/grimsel/core/autocomplete.py:167: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  pd.concat([self._df, self.df_add[add_cols]]))


done. Added: AT_TRNS_ST, DE_TRNS_RV, DE_TRNS_ST, AT_TRNS_RV, IT_TRNS_ST, IT_TRNS_RV, FR_TRNS_RV, FR_TRNS_ST, CH_TRNS_ST, CH_TRNS_RV
Autocompletion df_def_plant in AutoCompletePlantDmnd ... done. Added: AT_DMND, AT_DMND_FLEX, CH_DMND, CH_DMND_FLEX, DE_DMND, DE_DMND_FLEX, FR_DMND, FR_DMND_FLEX, IT_DMND, IT_DMND_FLEX
Autocompletion df_def_plant in AutoCompletePlantCons ... nothing added.
Autocompletion df_plant_encar in AutoCompletePpCaFlex ... done. Added: [77, 0], [79, 0], [81, 0], [83, 0], [85, 0]
############################################################
Writing table profinflow to output schema.
Writing table profsupply to output schema.
Writing table plant_encar to output schema.
Writing table hydro to output schema.
Writing table def_fuel to output schema.
Writing table plant_month to output schema.
Writing table fuel_node_encar to output schema.
Writing table def_month to output schema.
Writing table def_week to output schema.
Writing table parameter_month to output schema.
Writ

- Energy capacity
Storage level rule
Calculation of yearly totals rules
Yearly ramping calculation rule
Yearly fuel consumption rule
Yearly charging rule
Yearly transmission send rule
Yearly transmission receive rule
Fuel constraint rule
Reservoir boundary conditions rule
Hydro minimum monthly generation as fraction of maximum monthly inflow
Calculation of monthly totals rule
Objective rule quadratic
Calculation of ramp rates rule
Calculation of absolute ramp rates rule
Supply rule
Symmetry of transmission rule
Profile rule variables
Fuel consumption calculation rule
OM VC calculation rule
CO2 VC calculation rule --> OBSOLETE: Linear cost included directly in objective!!
Ramp VC calculation rule
Fixed O&M cost calculation rule
Fixed capital cost calculation rule
Initializing output table  var_sy_pwr
Initializing output table  var_sy_dmnd_flex
Initializing output table  var_sy_erg_st
Initializing output table  var_mt_erg_mt
Initializing output table  var_yr_erg_yr
Initializing output ta

## Some filtering of the ml.df_def_loop table so we don't end up doing 376650 model runs:

In [6]:

# figure 8/9: various nuclear power indicators france 
slct_vr = [0] + list(np.arange(0, ml.df_def_loop.swvr_id.max(), 4) + 1)
slct_st = list(np.arange(0, ml.df_def_loop.swvr_id.max() + 10, 10))
mask_base = (ml.df_def_loop.swvr_id.isin(slct_vr) &
               ml.df_def_loop.swst_id.isin(slct_st) &
               ml.df_def_loop.swtc_id.isin([0, 1]) &
               ml.df_def_loop.swpt_id.isin([0]) &
               ml.df_def_loop.swyr_id.isin([0]) &
               ml.df_def_loop.swco_id.isin([0]) &
               ml.df_def_loop.swrc_id.isin([0]))

# figure 10: ramping costs
slct_vr = list(np.arange(13, ml.df_def_loop.swvr_id.max() + 1, 4))
slct_st = list(np.arange(0, ml.df_def_loop.swvr_id.max() + 10, 10))
mask_ramping = (ml.df_def_loop.swvr_id.isin(slct_vr) &
                ml.df_def_loop.swst_id.isin(slct_st) &
                ml.df_def_loop.swtc_id.isin([0, 1]) &
                ml.df_def_loop.swpt_id.isin([0]) &
                ml.df_def_loop.swyr_id.isin([0]) &
                ml.df_def_loop.swco_id.isin([0])# &
#               ml.df_def_loop.swrc_id.isin([0]) --> all
               )

# figure 11: qualitatively diverging storage impact
slct_vr = [{round(val * 10000) / 10000 if not val == 'default' else val: key
            for key, val in dict_vre.items()}[vr] for vr in [0.5, 0.7, 0.4]]
mask_years = (ml.df_def_loop.swvr_id.isin(slct_vr) &
#                ml.df_def_loop.swst_id.isin(slct_st) & --> all
                ml.df_def_loop.swtc_id.isin([0, 1]) &
                ml.df_def_loop.swpt_id.isin([0]) &
#                ml.df_def_loop.swyr_id.isin([0]) & --> all
                ml.df_def_loop.swco_id.isin([0]) &
                ml.df_def_loop.swrc_id.isin([0]) 
                )

# figure 12: consecutive replacement
slct_vr = [0] + list(np.arange(0, ml.df_def_loop.swvr_id.max(), 4) + 1)
slct_st = list(np.arange(0, ml.df_def_loop.swvr_id.max() - 5, 10))
mask_consec = (ml.df_def_loop.swvr_id.isin(slct_vr) &
                ml.df_def_loop.swst_id.isin(slct_st) &
                ml.df_def_loop.swtc_id.isin([0]) &
                ml.df_def_loop.swpt_id.isin([0]) &
                ml.df_def_loop.swyr_id.isin([0]) &
                ml.df_def_loop.swco_id.isin([0, 1]) &
                ml.df_def_loop.swrc_id.isin([0]) 
                )

# figure 13: emissions
slct_vr = [0] + list(np.arange(0, ml.df_def_loop.swvr_id.max(), 2) + 1)
slct_st = list(np.arange(0, ml.df_def_loop.swvr_id.max() - 5, 10))
mask_emissions = (ml.df_def_loop.swvr_id.isin(slct_vr) &
                ml.df_def_loop.swst_id.isin(slct_st) &
                ml.df_def_loop.swtc_id.isin([0]) &
                ml.df_def_loop.swpt_id.isin([0]) &
                ml.df_def_loop.swyr_id.isin([0]) &
#                ml.df_def_loop.swco_id.isin([0]) & --> all
                ml.df_def_loop.swrc_id.isin([0]) 
                )

mask_total = mask_base | mask_ramping | mask_years | mask_consec | mask_emissions

ml.df_def_loop = ml.df_def_loop.loc[mask_total]

print(ml.df_def_loop[[c for c in ml.df_def_loop.columns if not any(sfx in c for sfx in ['_vl', '_id'])]])

      swvr  swst  swtc  swpt  swyr  swco  swrc
0      0.0   0.0   0.0   0.0   0.0   0.0   0.0
1      1.0   0.0   0.0   0.0   0.0   0.0   0.0
2      3.0   0.0   0.0   0.0   0.0   0.0   0.0
3      5.0   0.0   0.0   0.0   0.0   0.0   0.0
4      7.0   0.0   0.0   0.0   0.0   0.0   0.0
5      9.0   0.0   0.0   0.0   0.0   0.0   0.0
6     11.0   0.0   0.0   0.0   0.0   0.0   0.0
7     13.0   0.0   0.0   0.0   0.0   0.0   0.0
8     15.0   0.0   0.0   0.0   0.0   0.0   0.0
9     17.0   0.0   0.0   0.0   0.0   0.0   0.0
10    19.0   0.0   0.0   0.0   0.0   0.0   0.0
11    21.0   0.0   0.0   0.0   0.0   0.0   0.0
12    23.0   0.0   0.0   0.0   0.0   0.0   0.0
13    25.0   0.0   0.0   0.0   0.0   0.0   0.0
14    27.0   0.0   0.0   0.0   0.0   0.0   0.0
15    29.0   0.0   0.0   0.0   0.0   0.0   0.0
16    17.0   1.0   0.0   0.0   0.0   0.0   0.0
17    21.0   1.0   0.0   0.0   0.0   0.0   0.0
18    29.0   1.0   0.0   0.0   0.0   0.0   0.0
19    17.0   2.0   0.0   0.0   0.0   0.0   0.0
20    21.0   

In [ ]:

# init ModelLoopModifier
mlm = model_loop_modifier.ModelLoopModifier(ml)

# starting row of loop
irow_0 = ml.io.resume_loop if ml.io.resume_loop else 0

irow = irow = 0

ml.m._limit_prof_to_cap()
ml.perform_model_run(zero_run=True)
# %
for irow in list(range(irow_0, len(ml.df_def_loop))):
    run_id = irow

    print('select_run')
    ml.select_run(run_id)

    ####
    print('set_co2_price')
    mlm.set_co2_price()

    ####
    print('set_winsol_year')
    mlm.set_winsol_year()

    ####
    print('select_vre_pp_types')
    slct_pp, slct_pt = mlm.select_vre_pp_types()

    ####
    print('scale_vre')
    mlm.scale_vre(slct_pt, dict_vre)

    ####
    print('set_ramping_cost')
    mlm.set_ramping_cost()

    ####
    print('select_storage_tech')
    mlm.select_storage_tech()

    ####
    print('set_storage_cap')
    mlm.set_storage_cap(dict_st)

    ############### RUN MODEL ###############
    print('fill_peaker_plants')
    ml.m.fill_peaker_plants(demand_factor=2)

    print('_limit_prof_to_cap')
    ml.m._limit_prof_to_cap()

    print('perform_model_run')
    ml.perform_model_run()

    for fn in [fn for fn in os.listdir('.') if 'ephemeral' in fn or 'def_loop' in fn]:
        os.remove(fn)

sqac = sql_analysis_comp.SqlAnalysisComp(sc_out=sc_out, db=db)
sqac.build_tables_plant_run_quick()
sqac.build_table_plant_run_tot_balance(from_quick=True)


************************************************************
ModelBase: Limiting chp profiles to cap_pwr_leg... ok, nothing changed.
************************************************************
***************************************************
** First run without investments and retirements **
***************************************************

          create_tempfile is monkey patched
          

          create_tempfile is monkey patched
          
Solver script file=ephemeralhedg.cplex.script
Solver log file: '/tmp/manual_log_file_replace_all.cplex.sol'
Solver solution file: '/tmp/manual_soln_file_replace_all_0.cplex.sol'
Solver problem files: ('ephemeraliihg.pyomo.lp',)

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.7.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2017.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a co

/mnt/data/Dropbox/SHARED_REPOS/grimsel/core/io.py:633: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  _df['run_id'] = self.run_id


Writing to db:  out_replace_all.var_sy_pwr... done in 1.009 sec
Writing to db:  out_replace_all.var_sy_dmnd_flex... done in 0.085 sec
Writing to db:  out_replace_all.var_sy_pwr... done in 0.144 sec
Writing to db:  out_replace_all.var_sy_erg_st... done in 0.221 sec
Writing to db:  out_replace_all.var_yr_erg_yr... done in 0.071 sec
Writing to db:  out_replace_all.var_yr_erg_fl_yr... done in 0.070 sec
Writing to db:  out_replace_all.var_yr_pwr_ramp_yr... done in 0.046 sec
Writing to db:  out_replace_all.var_yr_vc_fl_pp_yr... done in 0.050 sec
Writing to db:  out_replace_all.var_yr_vc_ramp_yr... done in 0.040 sec
Writing to db:  out_replace_all.var_yr_vc_co2_pp_yr... done in 0.032 sec
Writing to db:  out_replace_all.var_yr_vc_om_pp_yr... done in 0.044 sec
Writing to db:  out_replace_all.var_yr_fc_om_pp_yr... done in 0.054 sec
Component fc_cp_pp_yr does not exist... skipping.
Writing to db:  out_replace_all.var_yr_fc_dc_pp_yr... done in 0.048 sec
Writing to db:  out_replace_all.var_yr_vc_dm

CPLEX> Solution written to file '/tmp/manual_soln_file_replace_all_1.cplex.sol'.
CPLEX> Writing to db:  out_replace_all.var_sy_pwr... done in 0.951 sec
Writing to db:  out_replace_all.var_sy_dmnd_flex... done in 0.088 sec
Writing to db:  out_replace_all.var_sy_pwr... done in 0.154 sec
Writing to db:  out_replace_all.var_sy_erg_st... done in 0.170 sec
Writing to db:  out_replace_all.var_yr_erg_yr... done in 0.052 sec
Writing to db:  out_replace_all.var_yr_erg_fl_yr... done in 0.047 sec
Writing to db:  out_replace_all.var_yr_pwr_ramp_yr... done in 0.038 sec
Writing to db:  out_replace_all.var_yr_vc_fl_pp_yr... done in 0.038 sec
Writing to db:  out_replace_all.var_yr_vc_ramp_yr... done in 0.037 sec
Writing to db:  out_replace_all.var_yr_vc_co2_pp_yr... done in 0.027 sec
Writing to db:  out_replace_all.var_yr_vc_om_pp_yr... done in 0.040 sec
Writing to db:  out_replace_all.var_yr_fc_om_pp_yr... done in 0.044 sec
Component fc_cp_pp_yr does not exist... skipping.
Writing to db:  out_replace_

   5   2.3642793e+11  -9.1326082e+10  3.61e+06  7.16e+06  8.56e+07
   6   1.3806477e+11   2.7293189e+09  2.08e+06  4.12e+06  4.93e+07
   7   8.7383726e+10   4.9404504e+10  8.13e+05  1.61e+06  1.93e+07
   8   7.7712291e+10   5.7749952e+10  4.62e+05  9.15e+05  1.09e+07
   9   7.2773003e+10   6.1821979e+10  2.39e+05  4.74e+05  5.67e+06
  10   6.9856628e+10   6.4700836e+10  1.02e+05  2.02e+05  2.41e+06
  11   6.8166007e+10   6.6633789e+10  2.35e+04  4.65e+04  5.56e+05
  12   6.7917359e+10   6.6904031e+10  1.09e+04  2.17e+04  2.59e+05
  13   6.7598795e+10   6.7351897e+10  5.40e+02  1.07e+03  1.28e+04
  14   6.7543762e+10   6.7448237e+10  4.58e+01  9.08e+01  1.09e+03
  15   6.7525693e+10   6.7481819e+10  1.14e+01  2.26e+01  2.70e+02
  16   6.7514767e+10   6.7501387e+10  3.32e+00  6.57e+00  7.86e+01
  17   6.7511419e+10   6.7507335e+10  1.32e-07  1.83e-10  5.73e-09
  18   6.7510034e+10   6.7509685e+10  4.17e-07  1.84e-10  5.67e-09
  19   6.7509924e+10   6.7509864e+10  9.29e-07  1.91e-10  5.57

...

Objective Q nonzeros : Min   : 0.01778243       Max   : 1084.591       
Linear constraints   :
  Nonzeros           : Min   : 0.02067500       Max   : 175000.0       
  RHS nonzeros       : Min   : 1.000000         Max   : 1.153685e+07   
CPLEX> CPXPARAM_Read_APIEncoding                        "UTF-8"
Tried aggregator 1 time.
QP Presolve eliminated 7653 rows and 4055 columns.
Aggregator did 443 substitutions.
Reduced QP has 945 rows, 2298 columns, and 4777 nonzeros.
Reduced QP objective Q matrix has 405 nonzeros.
Presolve time = 0.02 sec. (5.32 ticks)
Parallel mode: using up to 8 threads for barrier.
Number of nonzeros in lower triangle of A*A' = 6345
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.99 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 8
  Rows in Factor            = 945
  Integer space required    = 2863
  Total non-zeros in factor = 27168
  Total FP ops to factor    = 1054560
 Itn      Primal Obj    


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.7.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2017.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/tmp/manual_log_file_replace_all.cplex.sol' open.
CPLEX> Problem 'ephemeralbefb.pyomo.lp' read.
Read time = 0.06 sec. (1.15 ticks)
CPLEX> Solution file '/tmp/manual_soln_file_replace_all_0.cplex.sol' read.
CPLEX> Problem name         : ephemeralbefb.pyomo.lp
Objective sense      : Minimize
Variables            :    6796  [Nneg: 5589,  Free: 1207,  Qobj: 405]
Objective nonzeros   :     610
Objective Q nonzeros :     405
Linear constraints   :    9041  [Less: 4866,  Greater: 465,  Equal: 3710]
  Nonzeros           :   26579
  RHS nonzeros       :    1375

Variables            : Min LB: 0.000000 

Writing to db:  out_replace_all.var_tr_erg_trm_sd_yr... done in 0.024 sec
Writing to db:  out_replace_all.var_yr_erg_yr... done in 0.017 sec
select_run
set_co2_price
set_winsol_year
select_vre_pp_types
scale_vre
set_ramping_cost
select_storage_tech
set_storage_cap
fill_peaker_plants
_limit_prof_to_cap
************************************************************
ModelBase: Limiting chp profiles to cap_pwr_leg... ok, nothing changed.
************************************************************
perform_model_run
************************************************************
run_id = 133 of 2094
swvr_vl = 30.00%
swst_vl = 30.00%
swtc_vl = CAS_STO
swpt_vl = WIN_ONS|WIN_OFF|SOL_PHO
swyr_vl = 2015
swco_vl = 40EUR/tCO2
swrc_vl = x0.0
************************************************************
Using warmstartfile:  /tmp/manual_soln_file_replace_all_1.cplex.sol
Using solutionfile:  /tmp/manual_soln_file_replace_all_0.cplex.sol

          create_tempfile is monkey patched
          

          cre

Writing to db:  out_replace_all.par_vc_ramp... done in 0.031 sec
Writing to db:  out_replace_all.par_st_lss_hr... done in 0.017 sec
Writing to db:  out_replace_all.par_st_lss_rt... done in 0.017 sec
Component hyd_erg_min does not exist... skipping.
Writing to db:  out_replace_all.par_inflowprof... done in 0.133 sec
Writing to db:  out_replace_all.par_chpprof... done in 0.083 sec
Writing to db:  out_replace_all.par_supprof... done in 0.221 sec
Writing to db:  out_replace_all.par_priceprof... done in 0.014 sec
Component week_ror_output does not exist... skipping.
Writing to db:  out_replace_all.par_dmnd... done in 0.082 sec
Writing to db:  out_replace_all.var_sy_pwr... done in 0.096 sec
Writing to db:  out_replace_all.par_erg_inp... done in 0.081 sec
Writing to db:  out_replace_all.par_erg_chp... done in 0.068 sec
Writing to db:  out_replace_all.par_capchnge_max... done in 0.012 sec
Writing to db:  out_replace_all.par_price_co2... done in 0.042 sec
Writing to db:  out_replace_all.par_vc_

Writing to db:  out_replace_all.var_mt_erg_mt... done in 0.249 sec
Component share_ws_set does not exist... skipping.
Component co2_int does not exist... skipping.
Writing to db:  out_replace_all.par_cap_pwr_leg... done in 0.044 sec
Writing to db:  out_replace_all.par_cap_trm_leg... done in 0.098 sec
Writing to db:  out_replace_all.par_cf_max... done in 0.029 sec
Writing to db:  out_replace_all.par_grid_losses... done in 0.015 sec
Component erg_max does not exist... skipping.
Component hyd_pwr_in_mt_max does not exist... skipping.
Component hyd_pwr_out_mt_min does not exist... skipping.
Writing to db:  out_replace_all.par_vc_dmnd_flex... done in 0.015 sec
Writing to db:  out_replace_all.par_factor_vc_fl_lin_0... done in 0.017 sec
Writing to db:  out_replace_all.par_factor_vc_fl_lin_1... done in 0.025 sec
Writing to db:  out_replace_all.par_factor_vc_co2_lin_0... done in 0.019 sec
Writing to db:  out_replace_all.par_factor_vc_co2_lin_1... done in 0.019 sec
Writing to db:  out_replace_al